In [3]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import *
import torch

from utils import load_and_preprocess

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib

In [4]:
%matplotlib tk
location_id = 329 # 330, 331
path = 'data/hystreet_fussgaengerfrequenzen_seit2021.csv'
df = load_and_preprocess(path, location_id)

df = df.loc[df['is_special_day'] == 0]


In [ ]:
df

In [65]:
plt.plot(pd.to_datetime(df['date']), df['pedestrians_count'])
df2 = df.loc[df['is_special_day'] == 1]
plt.plot(pd.to_datetime(df2['date']), df2['pedestrians_count'])

In [67]:

np.sum(df['is_special_day'])

df[df['is_special_day']==1]

,temperature,pedestrians_count,is_holiday,is_special_day,date,weather_condition_clear-day,weather_condition_clear-night,weather_condition_cloudy,weather_condition_fog,weather_condition_partly-cloudy-day,weather_condition_partly-cloudy-night,weather_condition_rain,weather_condition_snow,weather_condition_wind,time_cos,time_sin,day_cos,day_sin,dayofyear_cos,dayofyear_sin


In [6]:
plt.plot(pd.to_datetime(df.date), df.pedestrians_count)

In [3]:
matplotlib.use("TkAgg")
plt.figure()
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df['date']], df['pedestrians_count'])
plt.show()

plt.figure()
plt.hist(df['pedestrians_count'], bins=50)
plt.show()

window_size = 24*365  # You can adjust this as needed
m_avg = df['pedestrians_count'].rolling(window=window_size, center=True).mean()

# Plot original data and centered moving average
plt.figure()
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df['date']], df['pedestrians_count'], label='Pedestrians Count')
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df['date']], m_avg, label='Moving Average', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Pedestrians Count')
plt.title('Pedestrians Count with Centered Moving Average')
plt.legend()
plt.show()

In [6]:
#split train-test manually
df_unseen = df[[datetime.strptime(d, '%Y-%m-%d %H:%M:%S') > datetime(2023, 5, 5) for d in df['date']]]
df_seen = df[[datetime.strptime(d, '%Y-%m-%d %H:%M:%S') <= datetime(2023, 5, 5) for d in df['date']]]


X_train = df_seen.drop(columns=["date", 'pedestrians_count', 'is_special_day'])
y_train = df_seen['pedestrians_count']

X_test = df_unseen.drop(columns=["date", 'pedestrians_count', 'is_special_day'])
y_test = df_unseen['pedestrians_count']

In [7]:
model = XGBRegressor()

params = dict()
params["device"] = "cuda" if torch.cuda.is_available() else "cpu"
params['eta'] = 0.1
params['verbosity'] = 2
params['objective'] = 'reg:squarederror'    
params['max_depth'] = 10
model.set_params(**params)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eta=0.2, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [8]:
print("Score in the training set:" , model.score(X_train, y_train))
print("Score in the test set:" , model.score(X_test, y_test))

Score in the training set: 0.998673605525999
Score in the test set: 0.9421187535257407


C:\Users\cemke\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [01:50:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [9]:
print(mean_squared_error(y_test, model.predict(X_test), squared=False))
print(mean_absolute_error(y_test, model.predict(X_test)))

356.6683315793036
207.43187075249512


In [10]:
print(mean_squared_error(y_train, model.predict(X_train), squared=False))
print(mean_absolute_error(y_train, model.predict(X_train)))

52.39612866973692
33.11129101548343


In [11]:
reals = np.array(df_unseen['pedestrians_count'])
preds = np.array(model.predict(df_unseen.drop(columns=['pedestrians_count', 'date', 'is_special_day'])))
dates = np.array([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_unseen['date']])

In [12]:
%matplotlib tk
matplotlib.use("TkAgg")
plt.figure()
plt.plot(dates, reals)
plt.plot(dates, preds)
plt.show()

In [14]:
errors = (reals-preds)
plt.hist(errors, bins=200)

(array([1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00, 1.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 1.000e+00, 2.000e+00, 0.000e+00, 3.000e+00, 2.000e+00,
        2.000e+00, 4.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 2.000e+00,
        1.000e+00, 1.000e+00, 3.000e+00, 2.000e+00, 0.000e+00, 4.000e+00,
        3.000e+00, 4.000e+00, 2.000e+00, 1.000e+00, 1.000e+00, 4.000e+00,
        0.000e+00, 5.000e+00, 3.000e+00, 6.000e+00, 4.000e+00, 3.000e+00,
        6.000e+00, 4.000e+00, 2.000e+00, 2.000e+00, 5.000e+00, 8.000e+00,
        6.000e+00, 3.000e+00, 1.000e+01, 7.000e+00, 1.900e+01, 1.000e+01,
        2.000e+01, 1.600e+01, 1.300e+01, 2.100e+01, 1.400e+01, 2.300e+01,
        1.900e+01, 3.400e+01, 3.600e+0

In [15]:
np.percentile(errors, 95)

605.4662109374999

In [16]:
np.percentile(errors, 5)

-358.349658203125

In [17]:
xgb.plot_importance(model, importance_type='weight', title='Feature importance by weight')
xgb.plot_importance(model, importance_type='gain', title='Feature importance by gain')
# xgb.plot_importance(model, importance_type='cover', title='Feature importance by cover')

<Axes: title={'center': 'Feature importance by gain'}, xlabel='F score', ylabel='Features'>

In [20]:
import shap

explainer = shap.Explainer(model)
shap_values = explainer(X_train, check_additivity=False)
shap.plots.beeswarm(shap_values, max_display=20)